In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install laserembeddings

In [ ]:
!python -m laserembeddings download-models

In [5]:
import pandas as pd
import time

from laserembeddings import Laser

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import random_split


In [133]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [134]:
#df = pd.read_csv("gdrive/MyDrive/Colab Notebooks/datathon2022/data/movie_data.csv")
#df.sample(5000).to_csv('gdrive/MyDrive/Colab Notebooks/datathon2022/data/movie_data_small.csv', index=False)
df = pd.read_csv("gdrive/MyDrive/Colab Notebooks/datathon2022/data/movie_data_small.csv")

In [137]:
class Movie(Dataset):
  def __init__(self, file, lang):
      self.df = pd.read_csv(file)
      mlingual_emb = Laser()

      self.data_embedding = mlingual_emb.embed_sentences(self.df['review'].tolist(), lang=lang)

  def __len__(self):
      return self.df.shape[0]

  def __getitem__(self, idx):
      text_emd = self.data_embedding[idx]
      label = self.df.iloc[idx, 1]
      return (text_emd, label)

In [138]:
training_data = Movie(file='gdrive/MyDrive/Colab Notebooks/datathon2022/data/movie_data_small.csv', lang='en')

In [139]:
train_size = round(len(training_data) * 0.8)
test_size = len(training_data) - train_size
train, val = random_split(training_data, [train_size, test_size])

In [140]:
len(train), len(val)

(4000, 1000)

In [141]:
train_loader = DataLoader(train, batch_size=64, shuffle=False)
val_loader = DataLoader(val, batch_size=64, shuffle=False)

In [142]:
x_batch, y_batch = next(iter(train_loader))

In [143]:
x_batch.shape, y_batch.shape

(torch.Size([64, 1024]), torch.Size([64]))

In [144]:
class Movie_NN(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.NN = nn.Sequential(
          nn.Linear(1024, 300),
          nn.PReLU(),
          nn.BatchNorm1d(300),
          nn.Dropout(0.4),
          nn.Linear(300, 128),
          nn.PReLU(),
          nn.BatchNorm1d(128),
          nn.Dropout(0.4),
          nn.Linear(128, 64),
          nn.PReLU(),
          nn.BatchNorm1d(64),
          nn.Linear(64, 1),
        )

    def forward(self, x):
        return self.NN(x)

# move the model to the GPU
model = Movie_NN()
model.to(device)

Movie_NN(
  (NN): Sequential(
    (0): Linear(in_features=1024, out_features=300, bias=True)
    (1): PReLU(num_parameters=1)
    (2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=300, out_features=128, bias=True)
    (5): PReLU(num_parameters=1)
    (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): PReLU(num_parameters=1)
    (10): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [145]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

In [146]:
def binary_acc(y_pred, y_true):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_true).sum().float()
    acc = correct_results_sum / y_true.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [149]:
def evaluate(model, val_loader, criterion):
  # goes through the test dataset and computes the test accuracy
  model.eval()  # bring the model into eval mode
  with torch.no_grad():
    acc_cum = 0.0
    val_loss_cum = 0.0
    num_samples_epoch = 0
    
    for X_batch, y_batch in val_loader:
        num_samples_batch = X_batch.shape[0]
        num_samples_epoch += num_samples_batch

        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch.unsqueeze(1).float())

        val_loss_cum += loss.item() * num_samples_batch
        acc_cum += binary_acc(y_pred, y_batch.unsqueeze(1)) * num_samples_batch
        
    avg_val_loss = val_loss_cum / num_samples_epoch
    avg_acc = acc_cum / num_samples_epoch
    assert 0 <= avg_acc <= 100

    return avg_acc, avg_val_loss

In [150]:
EPOCHS = 10
for e in range(1, EPOCHS+1):
    model.train()
    # reset statistics trackers
    train_loss_cum = 0.0
    acc_cum = 0.0
    num_samples_epoch = 0
    t = time.time()

    i = 0
    for X_batch, y_batch in train_loader:
        num_samples_batch = X_batch.shape[0]
        num_samples_epoch += num_samples_batch

        optimizer.zero_grad()
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch.unsqueeze(1).float())
        
        loss.backward()
        optimizer.step()

        # keep track of train stats
        train_loss_cum += loss.item() * num_samples_batch
        acc_cum += binary_acc(y_pred, y_batch.unsqueeze(1)) * num_samples_batch
        print(i, ' ', end='')
        i += 1
        
    # average the accumulated statistics
    avg_train_loss = train_loss_cum / num_samples_epoch
    avg_acc = acc_cum / num_samples_epoch
    validation_acc, validation_loss = evaluate(model, val_loader, criterion)
    epoch_duration = time.time() - t
        
    print()
    print(f'Epoch: {e} |'
          f'Train loss: {avg_train_loss:.4f} |'
          f'Train accuracy: {avg_acc.item():.4f} |'
          f'Validation loss: {validation_loss:.4f}|'
          f'Validation acc: {validation_acc:.4f} |'
          f'Duration: {epoch_duration:.2f} sec')

0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  
Epoch: 1 |Train loss: 0.5738 |Train accuracy: 69.6800 |Validation loss: 0.6051|Validation acc: 63.7600 |Duration: 0.69 sec
0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  
Epoch: 2 |Train loss: 0.4079 |Train accuracy: 81.8480 |Validation loss: 0.4513|Validation acc: 77.3120 |Duration: 1.06 sec
0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  
Epoch: 3 |Train loss: 0.3

In [151]:
# save model
torch.save(
    {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    },
    'gdrive/MyDrive/Colab Notebooks/datathon2022/data/model.'
)

# Predictions

In [152]:
sigmoid = nn.Sigmoid()
laser = Laser()

In [153]:
def predict(input_txt):
  text = laser.embed_sentences(input_txt, lang='de')
  text = torch.from_numpy(text)
  text = text.to(device)

  model.eval()
  return sigmoid(model(text))

In [154]:
predict("Der Film war sehr schlecht. Es macht keinen Sinn sich diesen anzusehen!")

tensor([[0.0007]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [155]:
predict("Der Film war sehr gut. Ich kann diesen sehr Empfehlen")

tensor([[0.9987]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [156]:
predict('Dieser Dokumentarfilm steckt voller Probleme.<br /><br />Wie arrogant ist es, einen Dokumentarfilm über die eigene Familie zu machen? Ich verstehe, dass Sie das Thema interessant finden. Ich war gelangweilt davon. Für mich ist das keine faszinierende Geschichte, und ich weiß nicht, warum Sie das denken sollten.<br /><br />Ich möchte nicht gemein rüberkommen, aber ich muss sagen: Die meisten Menschen in diesem Film sind einfach nicht attraktiv. Und das ist in Ordnung, nicht jeder ist hübsch. Aber deine Kameratechnik, ihnen das Objektiv ins Gesicht zu stecken, damit du nicht anders kannst, als von ihrem unangenehmen Aussehen überwältigt zu werden, weil es deinen 47-Zoll-Fernseher ausfüllt, ist nicht angenehm. Ich musste meine Hand die Hälfte der Zeit dafür heben schütze mich vor Warzen, Falten, Tränensäcken und gelben Zähnen. Wirklich, ich versuche, nicht unmenschlich zu klingen, aber ziehe die Kamera zurück, damit es nicht so ist, als würden mir völlig Fremde ins Gesicht atmen.<br /><br />Die Kameraführung in diesem "Film" ist auf Amateurniveau. Es ist die Art von Kameraführung, die man von jedem mit einem Camcorder bei einem Familienpicknick sieht. Uninteressante Bildeinstellungen, wackelige, sogar statische Aufnahmen werden nachlässig gemacht. Put a wenig Aufwand, wenn Sie damit Ihren Lebensunterhalt verdienen wollen.<br /><br />Ich verstehe ehrlich gesagt nicht, was die große Sache an dieser Sache sein soll.')

tensor([[0.0069]], device='cuda:0', grad_fn=<SigmoidBackward0>)

# German Test Set

In [157]:
test_data = Movie(file='gdrive/MyDrive/Colab Notebooks/datathon2022/data/movie_data_test_german.csv', lang='de')

In [158]:
test_loader = DataLoader(test_data, batch_size=50, shuffle=False)

In [159]:
evaluate(model, test_loader, criterion)

(tensor(84., device='cuda:0'), 0.33490899205207825)